In [1]:
from qiskit import (
    QuantumCircuit,
    QuantumRegister,
    ClassicalRegister,
    AncillaRegister,
    IBMQ,
    Aer,
    __qiskit_version__,
)
from qiskit.circuit.library import QFT, GraphState, GroverOperator
from qiskit.qasm import pi
from qiskit_nature.drivers import Molecule

import numpy as np
import matplotlib.pyplot as plt
import os.path
import time

from src.utils import *
from src.benchmarks import (
    grover,
    qaoa,
    ghz,
    qft,
    qftentangled,
    shor,
    wstate,
    qpeinexact,
    qpeexact,
    graphstate,
    vqe,
    dj,
    hhl,
    ae,
    qwalk,
)
from src.benchmarks.qiskit_application_optimization import routing, tsp
from src.benchmarks.qiskit_application_finance import (
    portfolioqaoa,
    portfoliovqe,
    pricingcall,
    pricingput,
)
from src.benchmarks.qiskit_application_ml import qgan
from src.benchmarks.qiskit_application_nature import groundstate, excitedstate

In [ ]:
ibm_native_gates = FakeMontreal().configuration().basis_gates
rigetti_native_gates = ["rx", "rz", "cz"]

In [ ]:
benchmarks = [
    graphstate,
    ghz,
    qft,
    wstate,
    qpeinexact,
    qpeexact,
    dj,
    qftentangled,
    ae,
    shor,
    qaoa,
    vqe,
    grover,
    qwalk,
    hhl,
]  # graphstate, ghz, qft, wstate, qpeinexact, qpeexact, dj, qftentangled, ae, shor,  qaoa, vqe,grover, qwalk, hhl]
benchmarks_application = [
    portfolioqaoa,
    portfoliovqe,
    qgan,
    groundstate,
]  # portfolioqaoa, portfoliovqe, pricingcall, pricingput, groundstate, excitedstate, routing, tsp, qgan
# benchmarks = []
# benchmarks_application = []

In [ ]:
n_min = 4
n_max = 6
stepsize = 2

save_hist = False
save_png = False

ancillary_mode = "v-chain"

In [ ]:
def create_shor_benchmarks():
    small = [15, 4]  # 18 qubits
    medium = [821, 4]  # 42 qubits
    large = [11777, 4]  # 58 qubits
    xlarge = [201209, 4]  # 74 qubits
    benchmark_list = [small, medium, large]  # , medium, large, xlarge]
    for n, a in benchmark_list:
        try:
            # Creating the circuit on Algorithmic Description Layer
            qc = shor.create_circuit(n, a)
            num_qubits = len(qc.qubits)

            # filename_algo, depth = handle_algorithm_layer(qc, num_qubits, save_png, save_hist)
            # characteristics.append([filename_algo, num_qubits, depth])

            # Creating the circuit on target-independent Description Layer
            filename_indep, depth = get_indep_layer(
                qc, num_qubits, save_png, save_hist, file_precheck=False
            )
            characteristics.append([filename_indep, num_qubits, depth])

            for gate_set, gate_set_name in gate_sets:
                try:

                    for opt_level in range(4):

                        # Creating the circuit on target-dependent: native gates layer
                        filename_transpiled, depth, n_actual = get_native_gates_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            num_qubits,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_transpiled, n_actual, depth])

                        # Creating the circuit on target-dependent: mapped layer for both mapping schemes
                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            False,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])

                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            True,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])
                except Exception as e:
                    print(
                        "\n Problem occured in inner loop: ",
                        "create_shor_benchmarks",
                        n,
                        gate_set_name,
                        e,
                    )
        except Exception as e:
            print("\n Problem occured in outer loop: ", "create_shor_benchmarks", n, e)
            continue

In [ ]:
def create_hhl_benchmarks(max_index: int):
    # n is not the number of qubits in this case
    for n in range(1, max_index):
        try:
            # Creating the circuit on Algorithmic Description Layer
            qc = hhl.create_circuit(n)
            num_qubits = len(qc.qubits)

            # filename_algo, depth = handle_algorithm_layer(qc, num_qubits, save_png, save_hist)
            # characteristics.append([filename_algo, num_qubits, depth])

            # Creating the circuit on target-independent Description Layer
            filename_indep, depth = get_indep_layer(
                qc, num_qubits, save_png, save_hist, file_precheck=False
            )
            characteristics.append([filename_indep, num_qubits, depth])

            for gate_set, gate_set_name in gate_sets:
                try:

                    for opt_level in range(4):

                        # Creating the circuit on target-dependent: native gates layer
                        filename_transpiled, depth, n_actual = get_native_gates_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            num_qubits,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_transpiled, n_actual, depth])

                        # Creating the circuit on target-dependent: mapped layer
                        # Creating the circuit on target-dependent: mapped layer for both mapping schemes
                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            False,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])

                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            True,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])
                except Exception as e:
                    print(
                        "\n Problem occured in inner loop: ",
                        "create_hhl_benchmarks",
                        n,
                        gate_set_name,
                        e,
                    )
        except Exception as e:
            print("\n Problem occured in outer loop: ", "create_hhl_benchmarks", n, e)
            continue

In [ ]:
def create_routing_benchmarks(max_index: int):
    # n is not the number of qubits in this case
    for n in range(2, max_index):
        try:
            # Creating the circuit on Algorithmic Description Layer
            qc = routing.create_circuit(n, 2)
            num_qubits = len(qc.qubits)

            # filename_algo, depth = handle_algorithm_layer(qc, num_qubits, save_png, save_hist)
            # characteristics.append([filename_algo, num_qubits, depth])

            # Creating the circuit on target-independent Description Layer
            filename_indep, depth = get_indep_layer(
                qc, num_qubits, save_png, save_hist, file_precheck=False
            )
            characteristics.append([filename_indep, num_qubits, depth])

            for gate_set, gate_set_name in gate_sets:
                try:

                    for opt_level in range(4):

                        # Creating the circuit on target-dependent: native gates layer
                        filename_transpiled, depth, n_actual = get_native_gates_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            num_qubits,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_transpiled, n_actual, depth])

                        # Creating the circuit on target-dependent: mapped layer for both mapping schemes
                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            False,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])

                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            True,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])
                except Exception as e:
                    print(
                        "\n Problem occured in inner loop: ",
                        "create_routing_benchmarks",
                        n,
                        gate_set_name,
                        e,
                    )
        except Exception as e:
            print(
                "\n Problem occured in outer loop: ", "create_routing_benchmarks", n, e
            )
            continue

In [ ]:
def create_tsp_benchmarks(max_index: int):
    # n is not the number of qubits in this case
    for n in range(3, max_index):
        try:
            # Creating the circuit on Algorithmic Description Layer
            qc = tsp.create_circuit(n)
            num_qubits = len(qc.qubits)

            # filename_algo, depth = handle_algorithm_layer(qc, num_qubits, save_png, save_hist)
            # characteristics.append([filename_algo, num_qubits, depth])

            # Creating the circuit on target-independent Description Layer
            filename_indep, depth = get_indep_layer(
                qc, num_qubits, save_png, save_hist, file_precheck=False
            )
            characteristics.append([filename_indep, num_qubits, depth])

            for gate_set, gate_set_name in gate_sets:
                try:

                    for opt_level in range(4):

                        # Creating the circuit on target-dependent: native gates layer
                        filename_transpiled, depth, n_actual = get_native_gates_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            num_qubits,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_transpiled, n_actual, depth])

                        # Creating the circuit on target-dependent: mapped layer for both mapping schemes
                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            False,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])

                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            True,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])
                except Exception as e:
                    print(
                        "\n Problem occured in inner loop: ",
                        "create_tsp_benchmarks",
                        n,
                        gate_set_name,
                        e,
                    )
        except Exception as e:
            print("\n Problem occured in outer loop: ", "create_tsp_benchmarks", n, e)
            continue

In [ ]:
def create_groundstate_benchmarks():

    m_1 = Molecule(
        geometry=[["Li", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 2.5]]],
        charge=0,
        multiplicity=1,
    )
    m_2 = Molecule(
        geometry=[["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.735]]],
        charge=0,
        multiplicity=1,
    )
    # m_3 = Molecule(geometry=[["O", [0.0, 0.0, 0.0]], ["H", [0.586, 0.757, .0]], ["H", [0.586, -0.757, .0]]], charge=0, multiplicity=1)

    molecules = [m_1, m_2]

    for count, molecule in enumerate(molecules):
        try:

            qc = groundstate.create_circuit(molecule)
            qc.name = qc.name + "-" + str(count)
            num_qubits = len(qc.qubits)

            # filename_algo, depth = handle_algorithm_layer(qc, num_qubits, save_png, save_hist)
            # characteristics.append([filename_algo, num_qubits, depth])

            # Creating the circuit on target-independent Description Layer
            filename_indep, depth = get_indep_layer(
                qc, num_qubits, save_png, save_hist, file_precheck=False
            )
            characteristics.append([filename_indep, num_qubits, depth])

            for gate_set, gate_set_name in gate_sets:
                try:

                    for opt_level in range(4):
                        # Creating the circuit on target-dependent: native gates layer
                        filename_transpiled, depth, n_actual = get_native_gates_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            num_qubits,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_transpiled, n_actual, depth])

                        # Creating the circuit on target-dependent: mapped layer for both mapping schemes
                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            False,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])

                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            True,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])
                except Exception as e:
                    print(
                        "\n Problem occured in inner loop: ",
                        "create_groundstate_benchmarks",
                        molecule,
                        gate_set_name,
                        e,
                    )
        except Exception as e:
            print(
                "\n Problem occured in outer loop: ",
                "create_groundstate_benchmarks",
                molecule,
                e,
            )
            continue

In [ ]:
def create_excitedstate_benchmarks():

    m_1 = Molecule(
        geometry=[["Li", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 2.5]]],
        charge=0,
        multiplicity=1,
    )
    m_2 = Molecule(
        geometry=[["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.735]]],
        charge=0,
        multiplicity=1,
    )
    # m_3 = Molecule(geometry=[["O", [0.0, 0.0, 0.0]], ["H", [0.586, 0.757, .0]], ["H", [0.586, -0.757, .0]]], charge=0, multiplicity=1)

    molecules = [m_1, m_2]

    for count, molecule in enumerate(molecules):
        try:
            qc = excitedstate.create_circuit(molecule)
            qc.name = qc.name + "-" + str(count)
            num_qubits = len(qc.qubits)

            # filename_algo, depth = handle_algorithm_layer(qc, num_qubits, save_png, save_hist)
            # characteristics.append([filename_algo, num_qubits, depth])

            # Creating the circuit on target-independent Description Layer
            filename_indep, depth = get_indep_layer(
                qc, num_qubits, save_png, save_hist, file_precheck=False
            )
            characteristics.append([filename_indep, num_qubits, depth])

            for gate_set, gate_set_name in gate_sets:
                try:

                    for opt_level in range(4):

                        # Creating the circuit on target-dependent: native gates layer
                        filename_transpiled, depth, n_actual = get_native_gates_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            num_qubits,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_transpiled, n_actual, depth])

                        # Creating the circuit on target-dependent: mapped layer for both mapping schemes
                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            False,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])

                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            True,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])
                except Exception as e:
                    print(
                        "\n Problem occured in inner loop: ",
                        "create_excitedstate_benchmarks",
                        molecule,
                        gate_set_name,
                        e,
                    )
        except Exception as e:
            print(
                "\n Problem occured in outer loop: ",
                "create_excitedstate_benchmarks",
                molecule,
                e,
            )
            continue

In [ ]:
def create_pricingcall_benchmarks(max_index: int):
    # n is not the number of qubits in this case
    for n in range(3, max_index):
        try:
            # Creating the circuit on Algorithmic Description Layer
            qc = pricingcall.create_circuit(n)
            num_qubits = str(len(qc.qubits))

            # filename_algo, depth = handle_algorithm_layer(qc, num_qubits, save_png, save_hist)
            # characteristics.append([filename_algo, num_qubits, depth])

            # Creating the circuit on target-independent Description Layer
            filename_indep, depth = get_indep_layer(
                qc, num_qubits, save_png, save_hist, file_precheck=False
            )
            characteristics.append([filename_indep, num_qubits, depth])

            for gate_set, gate_set_name in gate_sets:
                try:

                    for opt_level in range(4):

                        # Creating the circuit on target-dependent: native gates layer
                        filename_transpiled, depth, n_actual = get_native_gates_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            num_qubits,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_transpiled, n_actual, depth])

                        # Creating the circuit on target-dependent: mapped layer for both mapping schemes
                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            False,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])

                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            True,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])
                except Exception as e:
                    print(
                        "\n Problem occured in inner loop: ",
                        "create_pricingcall_benchmarks",
                        n,
                        gate_set_name,
                        e,
                    )
        except Exception as e:
            print(
                "\n Problem occured in outer loop: ",
                "create_pricingcall_benchmarks",
                n,
                e,
            )
            continue

In [ ]:
def create_pricingput_benchmarks(max_index: int):
    # n is not the number of qubits in this case
    for n in range(3, max_index):
        try:
            # Creating the circuit on Algorithmic Description Layer
            qc = pricingput.create_circuit(n)
            num_qubits = str(len(qc.qubits))

            # filename_algo, depth = handle_algorithm_layer(qc, num_qubits, save_png, save_hist)
            # characteristics.append([filename_algo, num_qubits, depth])

            # Creating the circuit on target-independent Description Layer
            filename_indep, depth = get_indep_layer(
                qc, num_qubits, save_png, save_hist, file_precheck=False
            )
            characteristics.append([filename_indep, num_qubits, depth])

            for gate_set, gate_set_name in gate_sets:
                try:
                    for opt_level in range(4):

                        # Creating the circuit on target-dependent: native gates layer
                        filename_transpiled, depth, n_actual = get_native_gates_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            num_qubits,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_transpiled, n_actual, depth])

                        # Creating the circuit on target-dependent: mapped layer for both mapping schemes
                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            False,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])

                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            True,
                            save_png,
                            save_hist,
                            file_precheck=False,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])
                except Exception as e:
                    print(
                        "\n Problem occured in inner loop: ",
                        "create_pricingput_benchmarks",
                        n,
                        gate_set_name,
                        e,
                    )
        except Exception as e:
            print(
                "\n Problem occured in outer loop: ",
                "create_pricingput_benchmarks",
                n,
                e,
            )
            continue

In [ ]:
characteristics = []
gate_sets = [(ibm_native_gates, "ibm"), (rigetti_native_gates, "rigetti")]

for benchmark in benchmarks + benchmarks_application:
    file_precheck = True
    print(benchmark)

    # First treat all "special" benchmarks with n != num qubits or different constructor parameters
    if benchmark == shor:
        # create_shor_benchmarks()
        continue
    elif benchmark == hhl:  # 5 is max number which fits my RAM
        create_hhl_benchmarks(2)
        continue
    elif benchmark == pricingcall:
        create_pricingcall_benchmarks(
            12
        )  # max val is not tested yet, but indep layer generation fails
        continue
    elif benchmark == pricingput:
        create_pricingput_benchmarks(
            12
        )  # max val is not tested yet, but indep layer generation fails
        continue
    elif benchmark == groundstate:
        create_groundstate_benchmarks()
        continue
    elif benchmark == excitedstate:
        create_excitedstate_benchmarks()
        continue
    elif benchmark == tsp:
        create_tsp_benchmarks(3)  # 5 is max number which fits my RAM
        continue
    elif benchmark == routing:
        create_routing_benchmarks(
            4
        )  # for a higher parameters than 5 it takes forever on my laptop
        continue

    for n in range(n_min, n_max, stepsize):
        try:
            start = time.time()
            # Creating the circuit on Algorithmic Description Layer
            if benchmark == grover or benchmark == qwalk:
                qc = benchmark.create_circuit(n, ancillary_mode=ancillary_mode)
                qc.name = qc.name + "-" + ancillary_mode
                file_precheck = False
                n = qc.num_qubits

            else:
                qc = benchmark.create_circuit(n)

            # filename_algo, depth = handle_algorithm_layer(qc, n, save_png, save_hist)
            # characteristics.append([filename_algo, n, depth])

            end = time.time()
            time_diff = end - start
            if time_diff > 60:
                print(qc.name + str(n) + " took too long")
                print(" time_diff: ", time_diff)
                break

            # Creating the circuit on target-independent Description Layer
            start = time.time()

            filename_indep, depth = get_indep_layer(
                qc, n, save_png, save_hist, file_precheck
            )
            characteristics.append([filename_indep, n, depth])

            end = time.time()
            time_diff = end - start
            if time_diff > 60 or depth > 1000000:
                print(filename_indep + " took too long or depth to large")
                print("depth: ", depth, " time_diff: ", time_diff)
                break

            for gate_set, gate_set_name in gate_sets:
                try:

                    for opt_level in range(4):

                        # Creating the circuit on target-dependent: native gates layer

                        start = time.time()

                        filename_transpiled, depth, n_actual = get_native_gates_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n,
                            save_png,
                            save_hist,
                            file_precheck=file_precheck,
                        )
                        characteristics.append([filename_transpiled, n_actual, depth])

                        end = time.time()
                        time_diff = end - start
                        if time_diff > 60 or depth > 1000000:
                            print(
                                filename_transpiled + " took too long or depth to large"
                            )
                            print("depth: ", depth, " time_diff: ", time_diff)
                            break

                        start = time.time()
                        # Creating the circuit on target-dependent: mapped layer for both mapping schemes
                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            False,
                            save_png,
                            save_hist,
                            file_precheck=file_precheck,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])

                        end = time.time()
                        time_diff = end - start
                        if time_diff > 60 or depth > 1000000:
                            print(filename_mapped + " took too long or depth to large")
                            print("depth: ", depth, " time_diff: ", time_diff)
                            break

                        start = time.time()

                        filename_mapped, depth = get_mapped_layer(
                            qc,
                            gate_set,
                            gate_set_name,
                            opt_level,
                            n_actual,
                            True,
                            save_png,
                            save_hist,
                            file_precheck=file_precheck,
                        )
                        characteristics.append([filename_mapped, n_actual, depth])

                        end = time.time()
                        time_diff = end - start
                        if time_diff > 60 or depth > 1000000:
                            print(filename_mapped + " took too long or depth to large")
                            print("depth: ", depth, " time_diff: ", time_diff)
                            break

                except Exception as e:
                    print(
                        "\n Problem occured in inner loop: ",
                        benchmark,
                        n,
                        gate_set_name,
                        e,
                    )
        except Exception as e:
            print("\n Problem occured in outer loop: ", benchmark, n, e)
            continue

# Generation of Benchmark Characteristics Graphs

In [ ]:
characteristics = np.array(characteristics)
plt.scatter(x=characteristics[:, 1].astype(int), y=characteristics[:, 2].astype(int))
plt.yscale("log")
plt.title("Depth and Width of generated Benchmarks")
plt.xlabel("# of Qubits")
plt.ylabel("Circuit Depth")
plt.savefig("test")

In [ ]:
len(characteristics)

In [ ]:
width_depth_0 = []
width_depth_1 = []
width_depth_2 = []
width_depth_3 = []

for f in characteristics:
    if "algorithm" in f[0]:
        width_depth_0.append([f[1], f[2]])

    elif "t-indep" in f[0]:
        width_depth_1.append([f[1], f[2]])

    elif "transpiled" in f[0]:
        width_depth_2.append([f[1], f[2]])

    elif "mapped" in f[0]:
        width_depth_3.append([f[1], f[2]])

In [ ]:
import random
import numpy
from matplotlib import pyplot

width_depth_0 = np.array(width_depth_0)
width_depth_1 = np.array(width_depth_1)
width_depth_2 = np.array(width_depth_2)
width_depth_3 = np.array(width_depth_3)

plt.scatter(
    x=width_depth_0[:, 0].astype(int), y=width_depth_0[:, 1].astype(int), label="0"
)
plt.yscale("log")
plt.title("Algorithm Layer")
plt.xlabel("# of Qubits")
plt.ylabel("Circuit Depth")
plt.xlim([0, 200])
plt.ylim([0, 10200])
plt.savefig("layer_0.pdf")
pyplot.show()
plt.scatter(
    x=width_depth_1[:, 0].astype(int), y=width_depth_1[:, 1].astype(int), label="1"
)
plt.yscale("log")
plt.title("Target-independent Layer")
plt.xlabel("# of Qubits")
plt.ylabel("Circuit Depth")
plt.xlim([0, 200])
plt.ylim([0, 10200])
plt.savefig("layer_1.pdf")
pyplot.show()
plt.scatter(
    x=width_depth_2[:, 0].astype(int), y=width_depth_2[:, 1].astype(int), label="2"
)
plt.yscale("log")
plt.title("Target-dependent Layer with native gates")
plt.xlabel("# of Qubits")
plt.ylabel("Circuit Depth")
plt.xlim([0, 200])
plt.ylim([0, 10200])
plt.savefig("layer_2.pdf")
pyplot.show()
plt.scatter(
    x=width_depth_3[:, 0].astype(int), y=width_depth_3[:, 1].astype(int), label="3"
)
plt.yscale("log")
plt.title("Target-independent Layer with native gates and mapping")
plt.xlabel("# of Qubits")
plt.ylabel("Circuit Depth")
plt.xlim([0, 200])
plt.ylim([0, 10200])
plt.savefig("layer_3.pdf")
pyplot.show();

Random debugging

In [3]:
qc = shor.create_circuit(5, 4)

# New Structure of Generator

In [3]:
def benchmark_generation_watcher(func, *args, max_depth:int, max_time:int):
    start = time.time()
    
    res = func(args)
    
    end = time.time()
    time_diff = end - start
    
    if max_time > 60:
        print("Calculation/Generation exceeded timeout limit.")
    if depth > max_depth:
        print("Depth of generated circuit is too large: ", depth)
        return False
        
    return res 

In [1]:
def create_benchmarks_from_config(config):
    benchmarks = config.benchmarks
    for benchmark in benchmarks:
        generate_benchmarks(benchmark.function, benchmark.parameters)

In [ ]:
def generate_scalable_benchmark(function, args):
     for n in range(n_min, n_max, stepsize):
        try:
            start = time.time()
            # Creating the circuit on Algorithmic Description Layer
            if benchmark == grover or benchmark == qwalk:
                qc = benchmark.create_circuit(n, ancillary_mode=ancillary_mode)
                qc.name = qc.name + "-" + ancillary_mode
                file_precheck = False
                n = qc.num_qubits

            else:
                qc = benchmark.create_circuit(n)
            
            end = time.time()
            time_diff = end - start
            if time_diff > 60:
                print(qc.name + str(n) + " took too long")
                print(" time_diff: ", time_diff)
                break
                
        except Exception as e:
            print("\n Problem occured in outer loop: ", benchmark, n, e)
            continue

            # filename_algo, depth = generate_algo_layer_circuit(qc, n, save_png, save_hist)
            # characteristics.append([filename_algo, n, depth])
            
            res_t_indep = generate_target_indep_layer_circuit()
            res_t_indep = generate_target_dep_layer_circuit()

                


In [ ]:
def generate_algo_layer_circuit(qc: QuantumCircuit, num_qubits:int):
    filename_algo, depth = benchmark_generation_watcher(handle_algorithm_layer, (
        qc, n, save_png, save_hist
    ))

    return filename_indep, depth   

In [ ]:
def generate_target_indep_layer_circuit(qc: QuantumCircuit, num_qubits:int):
    filename_indep, depth = benchmark_generation_watcher(get_indep_layer, (
        qc, n, save_png, save_hist, file_precheck
    ))

    return filename_indep, depth   

In [ ]:
def generate_target_dep_layer_circuit(qc: QuantumCircuit, num_qubits:int, gate_sets, save_png, save_hist, file_precheck)::
    for gate_set, gate_set_name in gate_sets:
        try:
            for opt_level in range(4):

                # Creating the circuit on target-dependent: native gates layer

                filename_transpiled, depth, n_actual = benchmark_generation_watcher(get_native_gates_layer, (
                    qc,
                    gate_set,
                    gate_set_name,
                    opt_level,
                    num_qubits,
                    True,
                    save_png,
                    save_hist,
                    file_precheck=file_precheck,
                ))
                characteristics.append([filename_transpiled, n_actual, depth])


                # Creating the circuit on target-dependent: mapped layer for both mapping schemes
                filename_mapped, depth = benchmark_generation_watcher(get_mapped_layer, (
                    qc,
                    gate_set,
                    gate_set_name,
                    opt_level,
                    n_actual,
                    False,
                    save_png,
                    save_hist,
                    file_precheck=file_precheck,
                ))


                filename_mapped, depth = benchmark_generation_watcher(get_mapped_layer, (
                    qc,
                    gate_set,
                    gate_set_name,
                    opt_level,
                    n_actual,
                    True,
                    save_png,
                    save_hist,
                    file_precheck=file_precheck,
                ))

                characteristics.append([filename_mapped, n_actual, depth])


            except Exception as e:
                print(
                    "\n Problem occured in inner loop: ",
                    benchmark,
                    n,
                    gate_set_name,
                    e,
                )
